In [1]:
import pandas as pd
import numpy as np
import ast
import nltk
import re
from bs4 import BeautifulSoup

def remove_space(string_list):
    k = [i.replace(' ', '') for i in string_list]
    return k

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

def clean_text(text):
    """
        text: a string
        return: modified initial string
    """
#     text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
#     text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
#     text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    return text

def to_list(inp):
    return "['"+inp+"']"
# Pass the matched technology column that we got
def convert_data_format(df,column_name):
    
    for i in range(0, len(df)):
        
        if df[column_name][i][0] == '[':
            res = ast.literal_eval(df[column_name][i])
            df[column_name][i] = res
            
        else:
            x = []
            x.insert(0, df[column_name][i])
            df[column_name][i] = x
            
    return df

def get_TP(df,manual_tagged_string,op_string):
    
    for ind in df.index:
        TP = FP = 0
        
        for i in range(0, len(df[op_string][ind])):
            
            if df[op_string][ind][i] in df[manual_tagged_string][ind]:
                TP = TP + 1
                
            else:
                pass
                
        tp.append(TP)

    return tp

def get_FP(df,manual_tagged_string, op_string):
    
    for ind in df.index:
        main_list = np.setdiff1d(df[op_string][ind],df[manual_tagged_string][ind])
        fp.append(len(main_list))
        fp_entry.append(main_list)
        
        
    return fp, fp_entry

def get_FN(df,manual_tagged_string, op_string):
    
    for ind in df.index:
        
        main_list = np.setdiff1d(df[manual_tagged_string][ind], df[op_string][ind])
        if df[manual_tagged_string][ind]=='[]':
            fn.append(0)
        else:
            fn.append(len(main_list))
        
    return fn

def get_tech_tagged_data(manually_tagged, tech_list):
    keep=[]
    for word in manually_tagged:
        if word in tech_list:
            keep.append(word)
    return keep
def combine(lis1, lis2):
    return list(set(lis1 + lis2))

def get_precision(tp,fp):
    
    if tp+fp ==0:
        precision = 0
    else:
        precision = tp/(tp+fp)
    return precision

def get_recall(tp,fn):
    if tp+fn ==0:
        recall = 0
    else:
        recall = tp/(tp+fn)
    return recall

def get_f_score(precision, recall):
    if precision+recall == 0:
        f_score = 0
    else:
        f_score = 2*(precision*recall)/(precision+recall)
    return f_score

def Diff(li1, li2): #FP entries
    return list(set(li2)-set(li1))

def Same(lis1,lis2): #TP entries
    return list(set(lis1).intersection(lis2))

def Common(lis1,lis2):
    return list(set(lis1)-(set(lis2)))

### ENG

In [8]:
##############################################################################################
data_path = "BERT_prod_tagsv2.csv" #ENTER DATA PATH
ID = 'lead_id'# ID column name
summaries = 'summaries'#summaries column name
manually_tagged = 'prod_tag' #manual tag column name
bert_tags = 'BERT_Tags'#column name
##############################################################################################

data=pd.read_csv(data_path)
# data=data[[ID,summaries,manually_tagged,bert_tags]]
data[manually_tagged]=data[manually_tagged].apply(clean_text)

data[manually_tagged]=data[manually_tagged].apply(eval)
data[bert_tags] = data[bert_tags].apply(eval)
data[bert_tags] = data[bert_tags].apply(remove_space)


tech_data = pd.read_csv("Tech_pydictionary_final.csv")
tech_data["Tech_word"]=tech_data["Tech_word"].apply(clean_text)
tech_list = list(set(tech_data["Tech_word"]))
eng_data = pd.read_csv("Eng_pydictionary_final.csv")
eng_data["English_word"]=eng_data["English_word"].apply(clean_text)
eng_list= list(set(eng_data["English_word"]))


# (all entries which are in eng dictionary)
eng_bert_tags=[]
removed=0
kept=0
for i in range(0,len(data)):
    eng_bert_1sen=[]
    tags= data[bert_tags][i]
    for j in tags:
        if j in eng_list:
            kept+=1
            eng_bert_1sen.append(j)
        else:
            removed+=1
    eng_bert_tags.append(eng_bert_1sen)
data[bert_tags+"_ENG"]=eng_bert_tags

##############################################################################################
data.to_csv("BERT_preds_JOB_id_ENG_ONLY2july.csv",index=False)
'''SAME FILE WRITE AND READ'''
summary = pd.read_csv("BERT_preds_JOB_id_ENG_ONLY2july.csv")
##############################################################################################
# summary = summary[[ID,summaries,manually_tagged,bert_tags]]
summary = summary.sort_values(by=[ID], ascending=False)
summary.reset_index(inplace = True, drop = True)
col_name_op = "BERT_Tags_ENG"

df_cm = summary
df_cm = convert_data_format(df_cm, col_name_op)
df_cm = convert_data_format(df_cm, manually_tagged)

count_context_technologies = []

for i in range(0, len(df_cm)):
    count_context_technologies.extend(df_cm[manually_tagged][i])
d = list(set(count_context_technologies))

tp = []
tn = []
fp = []
fn = []
TP = TN = FP = FN = 0
df_cm[manually_tagged] = df_cm[manually_tagged].fillna('[]')

df_cm['Eng_Label']=df_cm[manually_tagged].copy()

tech_tag_list = []

df_cm['Eng_Label'] = df_cm.apply(
    lambda x: get_tech_tagged_data(x["Eng_Label"], eng_list),
    axis = 1)
list_tech_dataset = []

# fill with 
for i in range(0, len(df_cm)):
    list_tech_dataset.extend(df_cm["Eng_Label"][i])

set_tech = set(list_tech_dataset)

for i in range(0, len(df_cm)):
    l = df_cm[col_name_op][i]
    df_cm[col_name_op][i] = [0 if element == '0' else element for element in l]

for i in range(0, len(df_cm)):
  words = df_cm[col_name_op][i]
  stopwords = [0]
  for word in list(words):
    if word in stopwords:
      words.remove(word)

# list of technologies which are not in extracted list but are in tagged list i.e. FN entries
df_cm['FP_entries_eng'] = df_cm.apply(lambda x: Diff(x["Eng_Label"], x[col_name_op]), axis = 1)

# list of technologies which are in extracted list and are in tagged list i.e. TP entries
df_cm['TP_entries_eng'] = df_cm.apply(lambda x: Same(x["Eng_Label"], x[col_name_op]), axis = 1)

# list of technologies which are NOT extracted list and are in tagged list i.e. FN entries
df_cm['FN_entries_eng'] = df_cm.apply(lambda x: Common(x["Eng_Label"], x[col_name_op]), axis = 1)

fp_entry =[]
tp = get_TP(df_cm, "Eng_Label", col_name_op)
fp, fp_entry_names = get_FP(df_cm, "Eng_Label", col_name_op)
fn = get_FN(df_cm, "Eng_Label", col_name_op)
# print(len(tp), len(fp), len(fn))
dict_fp = {}

for i in range(0, len(fp_entry_names)):
    fp_entry_names[i] = fp_entry_names[i].tolist()
    dict_fp[i] = fp_entry_names[i]

# creating a dataframe
df_matrix = pd.DataFrame(list(zip(tp, fp, fn)), 
               columns =['TP', 'FP', "FN"])
df_matrix['Precision'] = df_matrix.apply(lambda x: get_precision(x["TP"], x["FP"]), axis = 1)

df_matrix["Recall"] = df_matrix.apply(lambda x: get_recall(x["TP"], x["FN"]), axis = 1)

df_matrix["f_score"] = df_matrix.apply(lambda x: get_f_score(x["Precision"], x["Recall"]), axis = 1)

final_data = df_cm.join(df_matrix)
final_data.to_csv("ACC-BERT_preds_JOB_id_ENG_ONLY2july.csv",index=False) ##########GIVE ENG ACC NAME############

<ipython-input-1-73c5e2b78628>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name][i] = res
<ipython-input-8-9326b1338a8c>:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cm[col_name_op][i] = [0 if element == '0' else element for element in l]


### TECH Acc

In [13]:
summary = pd.read_csv(data_path)

# summary = summary[[ID,summaries,manually_tagged]]
summary[manually_tagged]=summary[manually_tagged].apply(clean_text)

summary = summary.sort_values(by=[ID], ascending=False)
summary.reset_index(inplace = True, drop = True)

# reading 
tech = pd.read_csv("Tech_pydictionary_final.csv")
col_name=tech.columns[-1]
tech[col_name] = tech[col_name].astype(str)
all_tech_words = list(tech[col_name].str.lower())
summary[summaries]=summary[summaries].astype(str)
all_strings = list(summary[summaries].str.lower())

i=1
tech_keys=[]
for item in all_strings:
#     print("***","TEST No,",i,"***")
    i=i+1
    tech_row=[]
    for k in all_tech_words:
        if k in item and len(k)>2:
           tech_row.append(k)
    tech_keys.append(tech_row)

summary['keywords']=tech_keys

def reg_match(txt, keyw):
    sep='|'
    temp = sep.join(['\\b' + i+'\\b' for i in keyw])
    matches = re.findall(temp,txt)
    if len(''.join(list(set(matches))))==0:
#         print(list(set(matches)))
        return []
    else:
        return list(set(matches))

summary['Tech_matched'] = summary.apply(
    lambda x: reg_match(x[summaries], x["keywords"]),
    axis = 1)

col_name_op = "Tech_matched"

df_cm = summary
df_cm = convert_data_format(df_cm, manually_tagged)
df_cm[manually_tagged] = df_cm[manually_tagged].fillna('[]')
count_context_technologies = []
for i in range(0, len(df_cm)):
    count_context_technologies.extend(df_cm[manually_tagged][i])
d = list(set(count_context_technologies))

tp = []
tn = []
fp = []
fn = []

TP = TN = FP = FN = 0

df_cm['Tech_Label'] = df_cm.apply(
    lambda x: get_tech_tagged_data(x[manually_tagged],tech_list), axis = 1)
string_match_tech_col = df_cm['Tech_Label']
list_tech_dataset = []

for i in range(0, len(df_cm)):
    list_tech_dataset.extend(df_cm["Tech_Label"][i])

set_tech = set(list_tech_dataset)

for i in range(0, len(df_cm)):
    l = df_cm[col_name_op][i]
    df_cm[col_name_op][i] = [0 if element == '0' else element for element in l]

for i in range(0, len(df_cm)):
  words = df_cm[col_name_op][i]
  stopwords = [0]
  for word in list(words):
    if word in stopwords:
      words.remove(word)
    
    
# list of technologies which are not in extracted list but are in tagged list i.e. FN entries
df_cm['FP_entries_TECH'] = df_cm.apply(lambda x: Diff(x["Tech_Label"], x[col_name_op]), axis = 1)

# list of technologies which are in extracted list and are in tagged list i.e. TP entries
df_cm['TP_entries_TECH'] = df_cm.apply(lambda x: Same(x["Tech_Label"], x[col_name_op]), axis = 1)

# list of technologies which are NOT extracted list and are in tagged list i.e. FN entries
df_cm['FN_entries_TECH'] = df_cm.apply(lambda x: Common(x["Tech_Label"], x[col_name_op]), axis = 1)
# function call
fp_entry =[]
tp = get_TP(df_cm, "Tech_Label", col_name_op)
fp, fp_entry_names = get_FP(df_cm, "Tech_Label", col_name_op)
fn = get_FN(df_cm, "Tech_Label", col_name_op)

# creating a dataframe
df_matrix = pd.DataFrame(list(zip(tp, fp, fn)), 
               columns =['TP', 'FP', "FN"])

df_matrix['Precision'] = df_matrix.apply(lambda x: get_precision(x["TP"], x["FP"]), axis = 1)

df_matrix["Recall"] = df_matrix.apply(lambda x: get_recall(x["TP"], x["FN"]), axis = 1)

df_matrix["f_score"] = df_matrix.apply(lambda x: get_f_score(x["Precision"], x["Recall"]), axis = 1)

df_cm=df_cm.drop(columns="keywords")
final_data = df_cm.join(df_matrix)
final_data.to_csv("ACC-TECH_BERT_preds_JOB_id2july.csv",index=False) ##########GIVE TECH ACC NAME############

<ipython-input-1-73c5e2b78628>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name][i] = res
<ipython-input-13-a44b83c069d2>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cm[col_name_op][i] = [0 if element == '0' else element for element in l]


In [14]:
all_tech_words

['visual basic.net', 'vmware', 'amazon ec2', 'symantec mail security']

### ENG+TECH

In [18]:
data=pd.read_csv("BERT_preds_JOB_id_ENG_ONLY2july.csv")#only eng   #####READ ENG ACC SHEET GENERATED ABOVE########
bert_tags="BERT_Tags_ENG"
col_name_op="BERT_ENG+string_matched"

data[bert_tags] = data[bert_tags].apply(str)
data[bert_tags] = data[bert_tags].apply(eval)
data[bert_tags] = data[bert_tags].apply(remove_space)
tech = pd.read_csv("Tech_pydictionary_final.csv")
col_name=tech.columns[-1]
summary=data
tech[col_name] = tech[col_name].astype(str)
all_tech_words = list(tech[col_name].str.lower())
summary[summaries]=summary[summaries].astype(str)
all_strings = list(summary[summaries].str.lower())
i=1
tech_keys=[]
for item in all_strings:
#     print("***","TEST No,",i,"***")
    i=i+1
    tech_row=[]
    for k in all_tech_words:
        if k in item and len(k)>2:
           tech_row.append(k)
    tech_keys.append(tech_row)
summary['keywords']=tech_keys

summary['string_matched'] = summary.apply(lambda x: reg_match(x[summaries], x["keywords"]), axis = 1)
summary['BERT_ENG+string_matched'] = summary.apply(
    lambda x: combine(x[bert_tags], x["string_matched"]), 
    axis = 1)

data=summary

########CHANGE NAME ##################
data.to_csv("BERT_preds_JOB_id_ENG+TECH2july.csv")
###########READ SAME FILE####################
summary = pd.read_csv("BERT_preds_JOB_id_ENG+TECH2july.csv")

col_name_op ="BERT_ENG+string_matched"#column name

summary = summary.sort_values(by=[ID], ascending=False)
summary.reset_index(inplace = True, drop = True)

df_cm = summary
df_cm = convert_data_format(df_cm, col_name_op)

df_cm = convert_data_format(df_cm,manually_tagged)

count_context_technologies = []

for i in range(0, len(df_cm)):
    count_context_technologies.extend(df_cm[manually_tagged][i])
d = list(set(count_context_technologies))

tp = []
tn = []
fp = []
fn = []

TP = TN = FP = FN = 0

df_cm[manually_tagged] = df_cm[manually_tagged].fillna('[]')

for i in range(0, len(df_cm)):
    l = df_cm[col_name_op][i]
    df_cm[col_name_op][i] = [0 if element == '0' else element for element in l]

for i in range(0, len(df_cm)):
  words = df_cm[col_name_op][i]
  stopwords = [0]
  for word in list(words):
    if word in stopwords:
      words.remove(word)
    

# list of technologies which are not in extracted list but are in tagged list i.e. FN entries
df_cm['FP_entries'] = df_cm.apply(lambda x: Diff(x[manually_tagged], x[col_name_op]), axis = 1)

# list of technologies which are in extracted list and are in tagged list i.e. TP entries
df_cm['TP_entries'] = df_cm.apply(lambda x: Same(x[manually_tagged], x[col_name_op]), axis = 1)

# list of technologies which are NOT extracted list and are in tagged list i.e. FN entries
df_cm['FN_entries'] = df_cm.apply(lambda x: Common(x[manually_tagged], x[col_name_op]), axis = 1)
fp_entry =[]

tp = get_TP(df_cm, manually_tagged, col_name_op)
fp, fp_entry_names = get_FP(df_cm, manually_tagged, col_name_op)
fn = get_FN(df_cm, manually_tagged, col_name_op)

dict_fp = {}

for i in range(0, len(fp_entry_names)):
    fp_entry_names[i] = fp_entry_names[i].tolist()
    dict_fp[i] = fp_entry_names[i]

# creating a dataframe
df_matrix = pd.DataFrame(list(zip(tp, fp, fn)), 
               columns =['TP', 'FP', "FN"])

df_matrix['Precision'] = df_matrix.apply(lambda x: get_precision(x["TP"], x["FP"]), axis = 1)

df_matrix["Recall"] = df_matrix.apply(lambda x: get_recall(x["TP"], x["FN"]), axis = 1)

df_matrix["f_score"] = df_matrix.apply(lambda x: get_f_score(x["Precision"], x["Recall"]), axis = 1)

final_data = df_cm.join(df_matrix)
final_data.to_csv("ACC-ENG+TECH_BERT_preds_JOB_id2july.csv",index=False) ##########GIVE ENG+TECH ACC NAME############

<ipython-input-1-73c5e2b78628>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name][i] = res
<ipython-input-18-8f74ca64818a>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cm[col_name_op][i] = [0 if element == '0' else element for element in l]
